In [1]:
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import re
import warnings

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [2]:
articles = pd.read_csv('articles.csv')
#print(articles.head())
#print("--")
customers = pd.read_csv('customers.csv')
print(customers.head())
print("--")
transactions = pd.read_csv("transactions_train.csv")
#print()

                                         customer_id   FN  Active   
0  00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...  NaN     NaN  \
1  0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...  NaN     NaN   
2  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...  NaN     NaN   
3  00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...  NaN     NaN   
4  00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...  1.0     1.0   

  club_member_status fashion_news_frequency   age   
0             ACTIVE                   NONE  49.0  \
1             ACTIVE                   NONE  25.0   
2             ACTIVE                   NONE  24.0   
3             ACTIVE                   NONE  54.0   
4             ACTIVE              Regularly  52.0   

                                         postal_code  
0  52043ee2162cf5aa7ee79974281641c6f11a68d276429a...  
1  2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...  
2  64f17e6a330a85798e4998f62d0930d14db8db1c054af6...  
3  5d36574f52495e81f019b680c843c443bd343d5ca5b

In [3]:
transactions_df = transactions

transactions_df = transactions_df.drop_duplicates(subset=['customer_id','article_id'])
transactions_df= transactions_df.reset_index(drop = True)
customer_list = transactions_df.customer_id.value_counts()[:50].index.tolist()
customers_1 = customers.loc[customers['customer_id'].isin(customer_list)]
transactions_train_t = transactions_df.loc[transactions_df['customer_id'].isin(customer_list)]
articles_t = articles.loc[articles['article_id'].isin(list(transactions_train_t['article_id']))]
merge_df = pd.DataFrame()
for customer in customers_1['customer_id']:
    new_df = pd.DataFrame()
    new_df['Buy'] = []
    #print(customer)
    x = [customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)
    #print(type(x))
    #new_df = new_df[x].reset_index(drop = True)
    new_df = pd.concat(x).reset_index(drop = True)#(pd.DataFrame(x)).reset_index(drop = True)
    print(new_df)

    #new_df = new_df.loc([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)

#     new_df = new_df.append([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)
    new_df['article_id'] = list(articles_t['article_id'])
    tran_temp = transactions_train_t.loc[transactions_train_t['customer_id']==customer]
    for article in list(tran_temp['article_id']):
        new_df.loc[(new_df['customer_id']==customer) & (new_df['article_id']==article),['Buy']] = 1

    #merge_df = pd.Series(new_df).reset_index(drop=True)
    merge_df = pd.concat([merge_df,new_df]).reset_index(drop = True)
merge_df

                                             customer_id   FN  Active   
0      03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0  \
1      03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
2      03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
3      03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
4      03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
...                                                  ...  ...     ...   
22919  03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
22920  03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
22921  03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
22922  03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
22923  03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   

      club_member_status fashion_news_frequency   age   
0                 ACTIVE              Regularly  51.0  \
1        

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,article_id,Buy
0,03d0011487606c37c1b1ed147fc72f285a50c05f00b971...,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,108775015,NaN
1,03d0011487606c37c1b1ed147fc72f285a50c05f00b971...,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,108775044,1.0
2,03d0011487606c37c1b1ed147fc72f285a50c05f00b971...,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,110065001,NaN
3,03d0011487606c37c1b1ed147fc72f285a50c05f00b971...,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,110065011,NaN
4,03d0011487606c37c1b1ed147fc72f285a50c05f00b971...,1.0,1.0,ACTIVE,Regularly,51.0,8db52856d17c197683efbc9d5ef2dc873aaf7062486b2d...,111565001,NaN
...,...,...,...,...,...,...,...,...,...
1146195,f137c16fd175271922dad4006565503952f24750a57388...,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,946795001,NaN
1146196,f137c16fd175271922dad4006565503952f24750a57388...,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,947509001,NaN
1146197,f137c16fd175271922dad4006565503952f24750a57388...,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,947934001,NaN
1146198,f137c16fd175271922dad4006565503952f24750a57388...,1.0,1.0,ACTIVE,Regularly,70.0,3455b39b24a47ae0262c91c5728ab9ddcfccc43628291e...,952267001,NaN


In [4]:
merge_df['Buy'] = merge_df['Buy'].fillna(0)
merge_df= pd.merge(merge_df, articles, on='article_id', how='left')
print(merge_df)

                                               customer_id   FN  Active   
0        03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0  \
1        03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
2        03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
3        03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
4        03d0011487606c37c1b1ed147fc72f285a50c05f00b971...  1.0     1.0   
...                                                    ...  ...     ...   
1146195  f137c16fd175271922dad4006565503952f24750a57388...  1.0     1.0   
1146196  f137c16fd175271922dad4006565503952f24750a57388...  1.0     1.0   
1146197  f137c16fd175271922dad4006565503952f24750a57388...  1.0     1.0   
1146198  f137c16fd175271922dad4006565503952f24750a57388...  1.0     1.0   
1146199  f137c16fd175271922dad4006565503952f24750a57388...  1.0     1.0   

        club_member_status fashion_news_frequency   age   
0                   ACTIVE              

In [5]:
x = merge_df.drop(columns=['Buy','FN','Active','customer_id','club_member_status','fashion_news_frequency', 'postal_code', 'prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name','section_name','garment_group_name','detail_desc','index_code']) 
y = merge_df['Buy']


# x = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
# y = np.array([1, 1, 2, 2])
#x = x.values.reshape(-1, 1)
#x = x.transpose()
#y = y.values.reshape(1,-1)
#y = y.transpose()

# print(x.shape)
# print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.8, random_state=42)
sgd = SGDClassifier(alpha=1, max_iter=10000)
sgd.fit(x,y)

SGDClassifier(alpha=1, max_iter=10000)

In [6]:
#print(type(x_test))
prediction = sgd.predict(x_test)

In [7]:
from sklearn.metrics import accuracy_score
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html


acc = accuracy_score(y_test, prediction)
print('Accuracy:', acc)

Accuracy: 0.9674794974699006


In [8]:
new_df = pd.DataFrame()
new_df['article_id'] = x_test['article_id']
new_df['model_output'] = prediction
print(new_df)

         article_id  model_output
957265    819113007           0.0
1056829   580102004           0.0
728664    828067002           0.0
746543    759108001           0.0
208701    581781010           0.0
...             ...           ...
856970    709187001           0.0
267942    797000001           0.0
942436    589722009           0.0
1061866   691072001           0.0
701195    764282001           0.0

[916960 rows x 2 columns]
